In [10]:
import os
from datetime import datetime
import numpy as np
from tensorflow.keras import layers
import tensorflow as tf
import numpy.matlib
from PIL import Image
from keras import backend as K
from scipy.special import softmax
import matplotlib.pyplot as plt
import matplotlib.patches as patches
# import Bottleneck

## MODEL
Define Bottleneck Residual layer for MobileNet using the same parameters as mentioned in the paper

In [34]:
class Bottleneck(tf.keras.Model):
  def __init__(self, expansion, filters, stride, block_id, alpha=1, ):
    super(Bottleneck, self).__init__(name="Bottleneck_"+block_id)
    self.expansion = expansion
    self.alpha = alpha
    self.output_channels = self.alpha*filters
    self.stride = stride
    self.out = None  # there was some problem with the eager execution

    prefix = 'Bottleneck_{}_'.format(block_id)
    self.prefix = prefix
    # expansion
    self.expand_BN = layers.BatchNormalization(name=prefix+'expand_BN')
    self.expand_ReLU = layers.ReLU(max_value=6, name=prefix+'expand_ReLU')
    # conv
    self.Conv = layers.DepthwiseConv2D(kernel_size=3, padding='same', strides=self.stride, use_bias=False, name=prefix+'conv')
    self.Conv_BN = layers.BatchNormalization(name=prefix+'conv_BN')
    self.Conv_ReLU = layers.ReLU(max_value=6, name=prefix+'conv_ReLU')
    # project
    self.project = layers.Conv2D(filters=self.output_channels, kernel_size=1, use_bias=False, name='contract')
    self.project_BN = layers.BatchNormalization(name=prefix+'contract_BN')
    # dimensions need to be the same for residual connection
    self.residual = layers.Add(name=prefix+'residual')

  def build(self, input_shape):
    self.d = input_shape[-1]
    self.expand = layers.Conv2D(filters=self.expansion*self.d, kernel_size=1, use_bias=False, name=self.prefix+'expand')

  def call(self, inputs):
    x = self.expand(inputs)
    x = self.expand_BN(x)
    x = self.expand_ReLU(x)
    self.out = x

    x = self.Conv(x)
    x = self.Conv_BN(x)
    x = self.Conv_ReLU(x)

    x = self.project(x)
    x = self.project_BN(x)

    if self.output_channels == self.d and self.stride == 1:
      x = self.residual([inputs, x])

    return x

  def model(self):
    x = tf.keras.Input(shape=(80, 80, 1))
    return tf.keras.Model(inputs=[x], outputs=self.call(x))

In [41]:
class MobileNetv2(tf.keras.Model):
  def __init__(self, k=11):
    super(MobileNetv2, self).__init__()
    self.k = k

    # self.pad = layers.ZeroPadding2D(padding=2, name='pad')
    self.conv_inp = layers.Conv2D(filters=32, kernel_size=3, strides=2, padding='same', activation='relu')
    self.BN = layers.BatchNormalization(name='BN')
    self.ReLU = layers.ReLU(max_value=6, name='ReLU')

    self.B1_1 = Bottleneck(expansion=1, filters=16, stride=1, block_id='B1_1')

    self.B2_1 = Bottleneck(expansion=6, filters=24, stride=2, block_id='B2_1')
    self.B2_2 = Bottleneck(expansion=6, filters=24, stride=1, block_id='B2_2')

    self.B3_1 = Bottleneck(expansion=6, filters=32, stride=2, block_id='B3_1')
    self.B3_2 = Bottleneck(expansion=6, filters=32, stride=1, block_id='B3_2')
    self.B3_3 = Bottleneck(expansion=6, filters=32, stride=1, block_id='B3_3')

    self.B4_1 = Bottleneck(expansion=6, filters=64, stride=2, block_id='B4_1')
    self.B4_2 = Bottleneck(expansion=6, filters=64, stride=1, block_id='B4_2')
    self.B4_3 = Bottleneck(expansion=6, filters=64, stride=1, block_id='B4_3')
    self.B4_4 = Bottleneck(expansion=6, filters=64, stride=1, block_id='B4_4')

    self.B5_1 = Bottleneck(expansion=6, filters=96, stride=1, block_id='B5_1')
    self.B5_2 = Bottleneck(expansion=6, filters=96, stride=1, block_id='B5_2')
    self.B5_3 = Bottleneck(expansion=6, filters=96, stride=1, block_id='B5_3')

    self.B6_1 = Bottleneck(expansion=6, filters=160, stride=2, block_id='B6_1')
    self.B6_2 = Bottleneck(expansion=6, filters=160, stride=1, block_id='B6_2')
    self.B6_3 = Bottleneck(expansion=6, filters=160, stride=1, block_id='B6_3')

    self.B7_1 = Bottleneck(expansion=6, filters=320, stride=1, block_id='B7_1')

    self.conv_out = layers.Conv2D(filters=1280, kernel_size=1, strides=(1, 1), use_bias=False, name='conv_out')
    self.avgpool = layers.AveragePooling2D(pool_size=(3, 3), name='avg_pool')
    self.conv_seg = layers.Conv2D(filters=self.k, kernel_size=1, strides=(1, 1), use_bias=False, name='conv_seg')

  def call(self, inputs):
    x = self.conv_inp(inputs)
    # x = self.BN(x)
    # x = self.ReLU(x)

    x = self.B1_1(x)

    x = self.B2_1(x)
    x = self.B2_2(x)

    x = self.B3_1(x)
    x = self.B3_2(x)
    x = self.B3_3(x)

    x = self.B4_1(x)
    x = self.B4_2(x)
    x = self.B4_3(x)
    x = self.B4_4(x)

    x = self.B5_1(x)
    x = self.B5_2(x)
    x = self.B5_3(x)

    x = self.B6_1(x)
    x = self.B6_2(x)
    x = self.B6_3(x)

    x = self.B7_1(x)

    x = self.conv_out(x)
    x = self.avgpool(x)
    c4 = self.conv_seg(x)

    return c4

  def model(self):
    x = tf.keras.Input(shape=(80, 80, 1))

    return tf.keras.Model(inputs=x, outputs=self.call(x))

In [42]:
# MOBILENET
MobileNetv2().model().summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 80, 80, 1)]       0         
                                                                 
 conv2d_5 (Conv2D)           (None, 40, 40, 32)        320       
                                                                 
 Bottleneck_B1_1 (Bottleneck  (None, 40, 40, 16)       2144      
 )                                                               
                                                                 
 Bottleneck_B2_1 (Bottleneck  (None, 20, 20, 24)       5568      
 )                                                               
                                                                 
 Bottleneck_B2_2 (Bottleneck  (None, 20, 20, 24)       9456      
 )                                                               
                                                           

In [25]:
class SSD(tf.keras.Model):
  def __init__(self, numBoxes=[4, 6, 6, 6, 4, 4], layerWidth=[28, 14, 7, 4, 2, 1], k=10+1+4):
    super(SSD, self).__init__()
    self.classes = k
    self.featureMaps = 6
    self.MobileNet = MobileNetv2(k=k)
    self.numBoxes = numBoxes
    self.layerWidth = layerWidth
    self.features = [None for _ in range(self.featureMaps)]
    self.classifiers = [None for _ in range(self.featureMaps)]

    self.conv1_1 = layers.Conv2D(256, 1, name='SSD_conv_1_1')
    self.conv1_2 = layers.Conv2D(512, 3, strides=(2, 2), padding='same', name='SSD_conv_1_2')

    self.conv2_1 = layers.Conv2D(128, 1, name='SSD_conv_2_1')
    self.conv2_2 = layers.Conv2D(256, 3, strides=(2, 2), padding='same', name='SSD_conv_2_2')

    self.conv3_1 = layers.Conv2D(128, 1, name='SSD_conv_3_1')
    self.conv3_2 = layers.Conv2D(256, 3, strides=(1, 1), name='SSD_conv_3_2')

    self.conv4_1 = layers.Conv2D(128, 1, name='SSD_conv_4_1')
    self.conv4_2 = layers.Conv2D(256, 2, strides=(1, 1), name='SSD_conv_4_2')  # changed kernel size to 2 since output of previous layer has width 3
    self.conv = []
    self.reshape = []

    for i in range(self.featureMaps):
      self.conv.append(layers.Conv2D(self.numBoxes[i]*self.classes, 3, padding='same', name='Classification_'+str(i)))
      self.reshape.append(layers.Reshape((self.layerWidth[i]*self.layerWidth[i]*self.numBoxes[i], self.classes), name='Reshape_classification_'+str(i)))

  def build(self, input_shape):
    self.MobileNet.build(input_shape)

  def call(self, inputs):
    x = inputs
    x = self.MobileNet(x)

    # get the convolved images at different resolutions
    self.features[0] = self.MobileNet.get_layer('Bottleneck_B4_1').out
    self.features[1] = self.MobileNet.get_layer('Bottleneck_B5_3').out
    self.features[2] = self.conv1_2(self.conv1_1(self.features[1]))
    self.features[3] = self.conv2_2(self.conv2_1(self.features[2]))
    self.features[4] = self.conv3_2(self.conv3_1(self.features[3]))
    self.features[5] = self.conv4_2(self.conv4_1(self.features[4]))

    for i in range(self.featureMaps):
      # for each feature map, create predictions according to the number of boxes for that layer and the number of output channels
      x = self.conv[i](self.features[i])
      x = self.reshape[i](x)
      self.classifiers[i] = x

    # concatenate all the classifiers
    x = layers.concatenate(self.classifiers, axis=-2, name='concatenate')
    return x

  def model(self):
    x = tf.keras.Input(shape=(80, 80, 1))
    return tf.keras.Model(inputs=x, outputs=self.call(x))

In [26]:
# # SSD
# NUM_CLASSES = 1
# # first 2 dimensions should be equal to width of output from bottleneck expand ReLU at the (4,1) and (5,3) respectively.
# # dimensions after second one are determined by convolutions written inside SSD (conv1_2, conv2_2, conv3_3, conv4_2)
# layerWidths = [28, 14, 7, 4, 2, 1]
# numBoxes = [3, 3, 3, 3, 3, 3]
# assert len(numBoxes) == len(layerWidths)  # numBoxes for each layer and each layer has a specific width
# outputChannels = NUM_CLASSES+1+4  # 1 classes + background + cx,cy,h,w
# assert outputChannels-NUM_CLASSES == 5

model = SSD(numBoxes=[3, 3, 3, 3, 3, 3], layerWidth=[28, 14, 7, 4, 2, 1], k=1+4)
model.model().summary()

TypeError: Exception encountered when calling layer "SSD_conv_1_1" (type Conv2D).

Originated from a graph execution error.

The graph execution error is detected at a node built at (most recent call last):
>>>  File /usr/lib/python3.8/runpy.py, line 194, in _run_module_as_main
>>>  File /usr/lib/python3.8/runpy.py, line 87, in _run_code
>>>  File /home/z/.local/lib/python3.8/site-packages/ipykernel_launcher.py, line 16, in <module>
>>>  File /home/z/.local/lib/python3.8/site-packages/traitlets/config/application.py, line 846, in launch_instance
>>>  File /home/z/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py, line 677, in start
>>>  File /home/z/.local/lib/python3.8/site-packages/tornado/platform/asyncio.py, line 199, in start
>>>  File /usr/lib/python3.8/asyncio/base_events.py, line 570, in run_forever
>>>  File /usr/lib/python3.8/asyncio/base_events.py, line 1859, in _run_once
>>>  File /usr/lib/python3.8/asyncio/events.py, line 81, in _run
>>>  File /home/z/.local/lib/py

## BOXES & IoU

In [ ]:
# I used less varying custom scales and aspect ratios here, since dataset is already uniform

#IMPORTANT: before changing the scales and aspect ratios, read the comment below
# number of scales is equal to number of different resolutions ie num of layer widths
# for a given resolution, we have different aspect ratios
# num(scales) = num(layerWidth) = num(numBoxes) and num(asp_ratios) = numBoxes[i]
MinScale = .1  # Min and Max scale given as percentage
MaxScale = 1.5
scales = [MinScale+x/len(layerWidths)*(MaxScale-MinScale) for x in range(len(layerWidths))]
scales = scales[::-1]  # reversing the order because the layerWidths go from high to low (lower to higher resoltuion)

asp = [0.5, 1.0, 1.5]
asp1 = [x**0.5 for x in asp]
asp2 = [1/x for x in asp1]
IMG_SIZE = 80
# should be equal to the 1st dimension in the output layer of the SSD model
BOXES = sum([a*a*b for a, b in zip(layerWidths, numBoxes)])
centres = np.zeros((BOXES, 2))
hw = np.zeros((BOXES, 2))
boxes = np.zeros((BOXES, 4))
print(BOXES)

In [ ]:
# calculating default box centres and height, width
idx = 0
for gridSize, numBox, scale in zip(layerWidths, numBoxes, scales):
  step_size = IMG_SIZE*1.0/gridSize
  for i in range(gridSize):
    for j in range(gridSize):
      pos = idx+(i*gridSize+j)*numBox
      # centre is the same for all aspect ratios(=numBox)
      centres[pos: pos+numBox, :] = i*step_size+step_size/2, j*step_size+step_size/2
      # height and width vary according to the scale and aspect ratio
      # zip asepct ratios and then scale them by the scaling factor
      hw[pos: pos+numBox, :] = np.multiply(gridSize*scale, np.squeeze(np.dstack([asp1, asp2]), axis=0))[:numBox, :]
  idx += gridSize*gridSize*numBox

# (x,y) co-ordinates of top left and bottom right
# This actually is not used anywhere. centres[] and hw[] are good enough substitute
boxes[:, 0] = centres[:, 0]-hw[:, 0]/2
boxes[:, 1] = centres[:, 1]-hw[:, 1]/2
boxes[:, 2] = centres[:, 0]+hw[:, 0]/2
boxes[:, 3] = centres[:, 1]+hw[:, 1]/2

In [ ]:
# calculate IoU for a set of search boxes and default boxes
def IoU(box1, box2):
  box1 = box1.astype(np.float64)
  box2 = box2.astype(np.float64)
  # find left and right co-ordinates of edges. Min should be less than Max for non zero overlap
  xmin = np.maximum(box1[:, 0], box2[:, 0])
  xmax = np.minimum(box1[:, 2], box2[:, 2])
  ymin = np.maximum(box1[:, 1], box2[:, 1])
  ymax = np.minimum(box1[:, 3], box2[:, 3])

  intersection = np.abs(np.maximum(xmax-xmin, 0)*np.maximum(ymax-ymin, 0))
  boxArea1 = np.abs((box1[:, 2]-box1[:, 0])*(box1[:, 3]-box1[:, 1]))
  boxArea2 = np.abs((box2[:, 2]-box2[:, 0])*(box2[:, 3]-box2[:, 1]))
  unionArea = boxArea1+boxArea2-intersection
  assert (unionArea > 0).all()
  iou = intersection/unionArea

  return iou


# give index of box correpsonding to IoUs > threshold (=0.5)
def bestIoU(searchBox):
  THRESHOLD = 0.5
  return np.argwhere(IoU(numpy.matlib.repmat(searchBox, BOXES, 1), boxes) > THRESHOLD)

## DATA

In [ ]:
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()










In [ ]:
# take mnist x and y pairs and convert to input, output pairs for the MobileNetv2+SSD model
def convert(x, y):
  MNIST_SIZE = x.shape[-1]
  # create a 2D array of top left corners for the mnist image to be placed
  corner = np.random.randint(IMG_SIZE-MNIST_SIZE, size=(x.shape[0], 2))

  # create a blank canvas for the input with the required dimension
  input = np.zeros((x.shape[0], IMG_SIZE, IMG_SIZE, 3))

  # replacing a part by RGB version of MNIST
  for i in range(x.shape[0]):
    lx = int(corner[i, 0])
    ly = int(corner[i, 1])
    input[i, lx:lx+MNIST_SIZE, ly:ly+MNIST_SIZE, :] = np.repeat(np.expand_dims(np.array(x[i, :, :]), axis=-1), 3, axis=-1)

  # for each default box, there are 5 values: class number and delta cx,cy,h,w
  output = np.zeros((y.shape[0], BOXES, 1+4))
  output[:, :, 0] = NUM_CLASSES  # defaulting class labels for all boxes to background initially
  for i in range(x.shape[0]):
    bbox = np.zeros(4)
    bbox[:2] = corner[i]
    bbox[2:] = corner[i]+(MNIST_SIZE, MNIST_SIZE)
    # for all default boxes which have IoU > threshold, set the delta values and class number
    box_idx = bestIoU(bbox).astype(np.uint16)
    output[i, box_idx, 0] = y[i]
    output[i, box_idx, 1] = (bbox[0]+bbox[2])/2.0-centres[box_idx, 0]
    output[i, box_idx, 2] = (bbox[1]+bbox[3])/2.0-centres[box_idx, 1]
    output[i, box_idx, 3] = MNIST_SIZE-hw[box_idx, 0]
    output[i, box_idx, 4] = MNIST_SIZE-hw[box_idx, 1]

  return input, output


In [ ]:
TRAINSIZE = 600
TESTSIZE = 100

x_train = x_train[:TRAINSIZE, :, :]
y_train = y_train[:TRAINSIZE]
x_test = x_test[:TESTSIZE, :, :]
y_test = y_test[:TESTSIZE]

test_x, test_y = convert(x_test, y_test)
train_x, train_y = convert(x_train, y_train)

In [ ]:
# checking if the inputs prepared are correct or not
r = np.random.randint(0, train_x.shape[0])
img = train_x[r, :, :, :].copy()
img_y = train_y[r]

im = np.array(Image.fromarray(img.astype(np.uint8)))
fig, ax = plt.subplots(1)
ax.imshow(im)

# find all boxes where class label is not background
idx = np.argwhere(img_y[:, 0] != NUM_CLASSES)[:, 0]
print('Number of boxes with IoU > 0.5:', idx.shape[0])
print('Green box: ground truth. Red box: default boxes with IoU > threshold')

#calculating the ground truth bounding boxes
gt = np.zeros(4, dtype=np.uint16)
gt[:2] = (img_y[idx[0], 1:3]+centres[idx[0], :2])
gt[2:] = (img_y[idx[0], 3:]+hw[idx[0], :])

# for some reason, x and y are inverted
rect = patches.Rectangle((gt[1]-gt[3]/2, gt[0]-gt[2]/2), gt[3], gt[2], linewidth=5, edgecolor='g', facecolor='none')
ax.add_patch(rect)

# showing all the boxes with IoU > 0.5
for i in idx:
  rect = patches.Rectangle((centres[i][1]-hw[i, 1]/2, centres[i][0]-hw[i, 0]/2), hw[i, 1], hw[i, 0], linewidth=1, edgecolor='r', facecolor='none')
  ax.add_patch(rect)

plt.show()

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y))

print(train_dataset.element_spec)
print(test_dataset.element_spec)

In [ ]:
BATCH_SIZE = 10
SHUFFLE_BUFFER_SIZE = 60
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)

## LOSS FUNCTION
Hard negative mining hasn't been done here
Initial idea was to assign weights to background classes, but there is some problem in that approach

In [ ]:
# label is not required here in the standard implementation
# calculate smooth L1 loss
def smoothL1(x, y, label):
  diff = K.abs(x-y)  #* K.switch(label == 10, label*1.0/BOXES, label)
  result = K.switch(diff < 1, 0.5*diff**2, diff-0.5)
  return K.mean(result)


def confidenceLoss(y, label):
  unweighted_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(label, y)
  # class_weights = tf.constant([[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0/BOXES]]*BOXES])
  # weights = tf.reduce_sum(class_weights * y, axis = -1)
  # weighted_loss = unweighted_loss * weights
  return K.mean(unweighted_loss)


def Loss(gt, y):
  # shape of y is n * BOXES * output_channels
  # shape of gt is n * BOXES * 5 
  loss = 0
  loss += smoothL1(y[:, :, -4:], gt[:, :, -4:], gt[:, :, 0:1])  # localisation loss
  loss += confidenceLoss(y[:, :, :-4], tf.cast(gt[:, :, 0], tf.int32))  # confidence loss
  return loss

## TRAIN

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001), loss=Loss)
history = model.fit(train_dataset, epochs=2, validation_data=test_dataset)

## SAVE

In [ ]:
model_name = "pplcntr_"+datetime.now().strftime("%Y%m%d-%H%M%S")
model.save(filepath=model_name, overwrite=True, include_optimizer=True, save_format='tf', signatures=None, options=None, save_traces=True, )
# model.save(model_name, save_format='tf')

# ## save weight
# model_weight = "pplcntr_" + dt.now().strftime("%Y%m%d-%H%M%S") + ".h5"
# model.save_weights(model_weight, save_format='h5')

In [ ]:
model.evaluate(test_x, test_y)

## INFERENCE

In [ ]:
# create sample data
X, Y = convert(x_test, y_test)

In [ ]:
# get prediction for one sample
y_pred = model.predict(X)
print(y_pred.shape)

In [ ]:
OBJperCLASS = 10  # get the top 10 results for each class


# get the confidence scores (with class values) and delta for the boxes. For each class, the top 10 values are used
def infer(Y):
  # classes are actually the index into the default boxes
  classes = np.zeros((OBJperCLASS, outputChannels-4), dtype=np.uint16)
  conf = np.zeros((OBJperCLASS, outputChannels-4))
  delta = np.zeros((OBJperCLASS, outputChannels-4, 4))
  class_predictions = softmax(Y[:, :outputChannels-4], axis=1)
  for i in range(outputChannels-4):
    classes[:, i] = Bottleneck.argpartition(class_predictions[:, i], BOXES-1-10, axis=-1)[-OBJperCLASS:]
    conf[:, i] = class_predictions[classes[:, i], i]
    delta[:, i] = Y[classes[:, i], outputChannels-4:]

  return conf, classes, delta


# generate bounding boxes from the inferred outputs
def Bbox(confidence, box_idx, delta):
  #delta contains delta(cx,cy,h,w)
  bbox_centre = np.zeros((OBJperCLASS, outputChannels-4, 2))
  bbox_hw = np.zeros((OBJperCLASS, outputChannels-4, 2))
  for i in range(OBJperCLASS):
    bbox_centre[i, :, 0] = centres[box_idx[i]][:, 0]+delta[i, :, 0]
    bbox_centre[i, :, 1] = centres[box_idx[i]][:, 1]+delta[i, :, 1]
    bbox_hw[i, :, 0] = hw[box_idx[i]][:, 0]+delta[i, :, 2]
    bbox_hw[i, :, 1] = hw[box_idx[i]][:, 1]+delta[i, :, 3]

  return bbox_centre, bbox_hw

In [ ]:
r = np.random.randint(TESTSIZE)

# top 10 predictions for each class
confidence, box_idx, delta = infer(y_pred[r])
bbox_centre, bbox_hw = Bbox(confidence, box_idx, delta)

im = np.array(Image.fromarray(X[r].astype(np.uint8)))
fig, ax = plt.subplots(1)
ax.imshow(im)

for i in range(outputChannels-4):
  # skipping backgrounds
  if i == NUM_CLASSES:
    continue
  color = 'r'
  # if a class is mentioned in the ground truth, color the boxes green
  if i in Y[r, :, 0]:
    color = 'g'
    print(i)

  # skip all the classes which have low confidence values
  if (confidence[:, i] > 0.5).any() or i in Y[r, :, 0]:
    for k in range(OBJperCLASS):
      print("{}: Confidence-{}\t\tCentre-{} Height,Width-{}".format(i, confidence[k, i], bbox_centre[k, i], bbox_hw[k, i]))

      # draw bounding box only if confidence scores are high
      if confidence[k, i] < 0.5:
        continue
      x = bbox_centre[k, i, 0]-bbox_hw[k, i, 0]/2
      y = bbox_centre[k, i, 1]-bbox_hw[k, i, 1]/2
      rect = patches.Rectangle((y, x), bbox_hw[k, i, 1], bbox_hw[k, i, 0], linewidth=1, edgecolor=color, facecolor='none')
      ax.add_patch(rect)

plt.show()